2024-09-24 - This was requested when Biswa asked to include more product type instead of only Stash. So created the data and provided the flag to Nayan<br>
2024-09-25 - Nayan replied that this only had the flag. He can also join the old data using customer id with this flag data.<br>
2024-09-25 - I am creating the entire data to be shared with Nayan again<br>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None) 

In [2]:
sq = """
with mau4m as 
(select customer_id, registration_date from prj-prod-dataplatform.worktable_data_analysis.customer_transaction_data
where 
    (tx_cnt_total_transactions_bw_3m_and_4m >=1  ---  Any one transaction greater than 100 pesos
    and 
    tx_cnt_active_loans_end_4m = 0)
    or
    ((tx_tsa_balance_4m+tx_stash_balance_4m) >= 100   ---    Total balance at the end of 5 month is >= 100 pesos
    and 
    tx_cnt_active_loans_end_4m = 0)
),
mau5m as
(select customer_id, registration_date from prj-prod-dataplatform.worktable_data_analysis.customer_transaction_data
where 
    (tx_cnt_total_transactions_bw_4m_and_5m >=1                     ---
    and                                                             ---     Any one transaction greater than 100 pesos         
    tx_cnt_active_loans_end_5m = 0)                                 ---  
    or                                                              ---    Total balance at the end of 5 month is >= 100 pesos
    ((tx_tsa_balance_5m+tx_stash_balance_5m) >= 100                 ---
    and 
    tx_cnt_active_loans_end_5m = 0)
),
lmt as 
(select customerId, startApplyDateTime, approvedDateTime from  `risk_credit_mis.loan_master_table`)
select 
ctd.customer_id,
ctd.registration_date,
ctd.tx_first_product,
case when ctd.customer_id in (select customer_id from mau4m) then 1 else 0 end tx_mau4m_active,
case when ctd.customer_id in (select customer_id from mau5m) then 1 else 0 end tx_mau5m_active,

---The user is Active (as per MAU definition) as of 120th Day from the Onboarding date

# case when ctd.tx_first_product in ('Individual Stash', 'Group Stash', 'Bills Pay', 'Unsecured Loan')  -- only Stash, billpay and Unsecured loans
#    then case when ctd.customer_id in (select customer_id from mau4m) --- Active MAU customers
#     then 0
#     else 1 
# end end tx_Churn_flag_4m,

# ---If the user is also Active (as per MAU definition) as of 150th Day from the Onboarding date then 0, otherwise 1

# case when ctd.tx_first_product in ('Individual Stash', 'Group Stash', 'Bills Pay', 'Unsecured Loan') then 
#    case when ctd.customer_id in (select customer_id from mau5m)
#     then 0
#     else 1
# end end tx_Churn_flag_5m,

# # ---The user is Active (as per MAU definition) as of 120th Day from the Onboarding date, AND did not submit any loan application to Tonik

case when ctd.tx_first_product in ('Individual Stash', 'Group Stash', 'Bills Pay' , 'TSA Top-Up', 'Time Deposit - 6 months', 'Time Deposit - 24 months'
                                    , 'Time Deposit - 12 months', 'Time Deposit - 9 months', 'Time Deposit - 18 months'
                                   ) then    ---- should be stash or bill payment
    case when ctd.customer_id in (select customer_id from mau4m)                                                                --- User is active MAU as of 120 day from Onboarding date
    and (select count(customerId) from `risk_credit_mis.loan_master_table` where customerId = cast(ctd.customer_id as numeric)
                                                               and date(startApplyDateTime) between date(ctd.registration_date)
                                                               and date_add(date(ctd.registration_date), interval 120 day)) = 0
    then 1
    else 0
end end tx_loanApplicationflag_4m,

(select count(customerId) from `risk_credit_mis.loan_master_table` where customerId = cast(ctd.customer_id as numeric)
                                                               and date(startApplyDateTime) between date(ctd.registration_date) 
                                                               and date_add(date(ctd.registration_date), interval 120 day)) as  submittedappliction4m,
                                                               
---If the users submitted any Cash loan application within 5th month (next 30 days from the 120th day from onboarding date) at Tonik then 1, otherwise 0

case when ctd.tx_first_product in ('Individual Stash', 'Group Stash', 'Bills Pay' , 'TSA Top-Up', 'Time Deposit - 6 months', 'Time Deposit - 24 months'
                                    , 'Time Deposit - 12 months', 'Time Deposit - 9 months', 'Time Deposit - 18 months') then
    case when ctd.customer_id in (select customer_id from mau5m)
    and(select count(customerId) from `risk_credit_mis.loan_master_table` where customerId = cast(ctd.customer_id as numeric)
                                                               and date(startApplyDateTime) between  date_add(ctd.registration_date, interval 120 day)
                                                               and date_add(ctd.registration_date, interval 150 day)) > 0
    then 1
    else 0
end end tx_loanApplicationflag_5m,

(select count(customerId) from `risk_credit_mis.loan_master_table` where customerId = cast(ctd.customer_id as numeric)
                                                               and date(startApplyDateTime) between  date_add(ctd.registration_date, interval 120 day)
                                                               and date_add(ctd.registration_date, interval 150 day)) as submittedappliction5m,
                                                               
---The user is Active (as per MAU definition) as of 120th Day from the Onboarding date, AND did not submit any loan application to Tonik within 120 days from onboarding date       
                                                        
case when ctd.tx_first_product in ('Individual Stash', 'Group Stash', 'Bills Pay' , 'TSA Top-Up', 'Time Deposit - 6 months', 'Time Deposit - 24 months'
                                    , 'Time Deposit - 12 months', 'Time Deposit - 9 months', 'Time Deposit - 18 months') then
    case when ctd.customer_id in (select customer_id from mau5m)
    and(select count(customerId) from `risk_credit_mis.loan_master_table` where customerId = cast(ctd.customer_id as numeric)
                                                               and date(approvedDateTime) between  date_add(ctd.registration_date, interval 120 day)
                                                               and date_add(ctd.registration_date, interval 150 day)
                                                            #    and date(approvedDateTime) is not null 
                                                               and flagApproval = 1
                                                               and new_loan_type in ('Quick', 'Flex')) > 0
    then 1
    else 0
end end tx_loanApprovedflag_5m,
---The user is Active (as per MAU definition) as of 120th Day from the Onboarding date AND never NPL AND current DPD status <=FSPD30 as of 120th day from the Onboarding date
# case when ctd.tx_first_product in ('Unsecured Loan') then
#      case when ctd.customer_id in (select customer_id from mau4m)
#      and (select count(customerId) from `risk_credit_mis.loan_master_table` where customerId = cast(ctd.customer_id as numeric)
#                                                                             and Max_Ever_DPD >= 180
#                                                                             and new_loan_type in ('Quick', 'Flex')) = 0
#      and (select count(lmt.customerId) from prj-prod-dataplatform.risk_credit_mis.loan_bucket_flow_report_core lbfrc 
#                    inner join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = lbfrc.loanAccountNumber
#                    inner join prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data ldd on ldd.loanAccountNumber = lmt.loanAccountNumber
#                    where lmt.customerId = cast(ctd.customer_id as numeric)
#                    and lbfrc.bucketDate = date_add(ctd.registration_date, interval 120 day)
#                    and ldd.min_inst_def30 in (1, 2)
#                    and lmt.new_loan_type in ('Quick', 'Flex')
#                   ) = 0
#      then 0
#      else 1
# end end tx_FSPD30_4m,
# (select count(customerId) from `risk_credit_mis.loan_master_table` where customerId = cast(ctd.customer_id as numeric)
#                                                                             and Max_Ever_DPD >= 180
#                                                                             and new_loan_type in ('Quick', 'Flex')) npl,
# ---If the user becomes FSPD30 within next 90 days from the observation date (120th day from the onboarding date) then 1; otherwise 0
# case when ctd.tx_first_product in ('Unsecured Loan') then
#      case when ctd.customer_id in (select customer_id from mau5m)
#      and (select count(lmt.customerId) from prj-prod-dataplatform.risk_credit_mis.loan_bucket_flow_report_core lbfrc 
#                    inner join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = lbfrc.loanAccountNumber
#                    inner join prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data ldd on ldd.loanAccountNumber = lmt.loanAccountNumber
#                    where lmt.customerId = cast(ctd.customer_id as numeric)
#                    and lbfrc.bucketDate = date_add(ctd.registration_date, interval 150 day)
#                    and ldd.min_inst_def30 in (1, 2)
#                    and lmt.new_loan_type in ('Quick', 'Flex')
#                   ) = 0
#      then 0
#      else 1
# end end tx_FSPD30_5m,
# (select count(lmt.customerId) from prj-prod-dataplatform.risk_credit_mis.loan_bucket_flow_report_core lbfrc 
#                    inner join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = lbfrc.loanAccountNumber
#                    inner join prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data ldd on ldd.loanAccountNumber = lmt.loanAccountNumber
#                    where lmt.customerId = cast(ctd.customer_id as numeric)
#                    and lbfrc.bucketDate = date_add(ctd.registration_date, interval 120 day)
#                    and ldd.min_inst_def30 in (1, 2)
#                    and lmt.new_loan_type in ('Quick', 'Flex')
#                   ) fspd304m,
# (select count(lmt.customerId) from prj-prod-dataplatform.risk_credit_mis.loan_bucket_flow_report_core lbfrc 
#                    inner join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = lbfrc.loanAccountNumber
#                    inner join prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data ldd on ldd.loanAccountNumber = lmt.loanAccountNumber
#                    where lmt.customerId = cast(ctd.customer_id as numeric)
#                    and lbfrc.bucketDate = date_add(ctd.registration_date, interval 150 day)
#                    and ldd.min_inst_def30 in (1, 2)
#                    and lmt.new_loan_type in ('Quick', 'Flex')
#                   ) fspd305m
from prj-prod-dataplatform.worktable_data_analysis.customer_transaction_data ctd
"""

df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')



Job ID 77a58146-8dbd-4b4a-a1b9-ad1b45c4ea16 successfully executed: |          |
Downloading: 100%|██████████|


In [3]:
df.columns

Index(['customer_id', 'registration_date', 'tx_first_product',
       'tx_mau4m_active', 'tx_mau5m_active', 'tx_loanApplicationflag_4m',
       'submittedappliction4m', 'tx_loanApplicationflag_5m',
       'submittedappliction5m', 'tx_loanApprovedflag_5m'],
      dtype='object')

In [4]:
df.shape

(473294, 10)

In [5]:
dfgroupby = df.groupby('tx_first_product').agg(customer_Unq_cnt = ('customer_id', 'nunique')).reset_index()
dfgroupby

,tx_first_product,customer_Unq_cnt
0,Bills Pay,280
1,Group Stash,1918
2,Home Loan- Secured Multipurpose,3
3,Individual Stash,22749
4,SIL ZERO,2
5,SIL ZERO VAS,5
6,SIL-Direct Settlement,23519
7,SIL-VAS,18783
8,Salary Backed Installment Loan - R - A,2098
9,Salary Backed Installment Loan - R - A - VAS,72


# Loan Application Flag

In [6]:
# Define the list of values to filter
a = ['Individual Stash', 'Group Stash', 'Bills Pay' , 'TSA Top-Up', 'Time Deposit - 6 months', 'Time Deposit - 24 months'
                                    , 'Time Deposit - 12 months', 'Time Deposit - 9 months', 'Time Deposit - 18 months']

# Filter the DataFrame for rows where 'tx_first_product' is in the list 'a'
df_filtered = df[df['tx_first_product'].isin(a)]

In [7]:
df_filtered.shape

(76930, 10)

In [8]:
df_filteredgroupby = df_filtered.groupby('tx_first_product').agg(customer_Unq_cnt = ('customer_id', 'nunique')).reset_index()
df_filteredgroupby

,tx_first_product,customer_Unq_cnt
0,Bills Pay,280
1,Group Stash,1918
2,Individual Stash,22749
3,TSA Top-Up,49127
4,Time Deposit - 12 months,74
5,Time Deposit - 18 months,13
6,Time Deposit - 24 months,114
7,Time Deposit - 6 months,2629
8,Time Deposit - 9 months,26


In [9]:
pd.set_option("display.max_rows", None)
df_filtered.groupby(['tx_first_product','tx_mau4m_active', 'tx_mau5m_active', 'tx_loanApplicationflag_4m','tx_loanApplicationflag_5m', 'tx_loanApprovedflag_5m']).agg(customer_unq_cnt = ('customer_id', 'nunique')).reset_index()

,tx_first_product,tx_mau4m_active,tx_mau5m_active,tx_loanApplicationflag_4m,tx_loanApplicationflag_5m,tx_loanApprovedflag_5m,customer_unq_cnt
0,Bills Pay,0,0,0,0,0,145
1,Bills Pay,0,1,0,0,0,6
2,Bills Pay,0,1,0,1,0,3
3,Bills Pay,1,0,0,0,0,20
4,Bills Pay,1,0,1,0,0,20
5,Bills Pay,1,1,0,0,0,25
6,Bills Pay,1,1,0,1,0,7
7,Bills Pay,1,1,1,0,0,53
8,Bills Pay,1,1,1,1,0,1
9,Group Stash,0,0,0,0,0,1490


In [10]:
df_filtered[df_filtered['tx_loanApplicationflag_4m']==0].groupby(['tx_first_product','tx_mau4m_active', 'tx_mau5m_active', 'tx_loanApplicationflag_4m','tx_loanApplicationflag_5m', 'tx_loanApprovedflag_5m']).agg(customer_unq_cnt = ('customer_id', 'nunique')).reset_index()

,tx_first_product,tx_mau4m_active,tx_mau5m_active,tx_loanApplicationflag_4m,tx_loanApplicationflag_5m,tx_loanApprovedflag_5m,customer_unq_cnt
0,Bills Pay,0,0,0,0,0,145
1,Bills Pay,0,1,0,0,0,6
2,Bills Pay,0,1,0,1,0,3
3,Bills Pay,1,0,0,0,0,20
4,Bills Pay,1,1,0,0,0,25
5,Bills Pay,1,1,0,1,0,7
6,Group Stash,0,0,0,0,0,1490
7,Group Stash,0,1,0,0,0,96
8,Group Stash,0,1,0,1,1,1
9,Group Stash,1,0,0,0,0,14


In [ ]:
df_filtered[df_filtered['tx_loanApplicationflag_4m']==0].to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Customer_360_Data_Prep\Data\FinalData\loanApplicationFlag_Customer360data_20240924.csv", index = False)

In [12]:
sq = """select * from prj-prod-dataplatform.worktable_data_analysis.customer_growth_combined_data"""

dfcombined = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 709cb1e8-750b-4ce7-be99-d48435ec3e1d successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [13]:
dfcombined.columns.values

array(['customer_id', 'onboarding_date', 'tx_first_product',
       'tx_first_product_user_segment', 'de_kyc_status',
       'de_educationdataflag', 'de_education_id', 'de_Education_Type',
       'de_clientdemographicflag', 'de_user_id', 'de_device_id',
       'de_created_dt', 'de_gender', 'de_birthplace', 'de_mobile_no',
       'de_product', 'de_type', 'de_telcoProvider', 'de_cdd_flag',
       'de_dateOfBirth', 'de_signUpAccNo', 'de_kycStatusfromCDD',
       'de_addressline1', 'de_city', 'de_province', 'de_barangay',
       'de_provinceCode', 'de_postalcode', 'de_mailingCity',
       'de_mailingPostalCode', 'de_mailingProvince', 'de_natureofwork',
       'de_employmentstatus', 'de_mobileOs', 'de_docType',
       'de_Minutestakentoonboard', 'de_companyName', 'de_industry',
       'de_device_dtl', 'de_Region_name', 'de_locationdfflag',
       'de_Onboarding_latitude', 'de_Onboarding_longitude',
       'de_loanjourney_latitude', 'de_loanjouney_longitude',
       'de_ApigeeLogs_latitude',

In [14]:
dfcombined.shape

(600939, 286)

In [15]:
duplicate_ids = dfcombined[dfcombined['customer_id'].duplicated()]['customer_id']
duplicate_ids

Series([], Name: customer_id, dtype: object)

In [16]:
df1 = dfcombined[['customer_id', 'onboarding_date', 'tx_first_product',
       'tx_first_product_user_segment', 'de_kyc_status',
       'de_educationdataflag', 'de_education_id', 'de_Education_Type',
       'de_clientdemographicflag', 'de_user_id', 'de_device_id',
       'de_created_dt', 'de_gender', 'de_birthplace', 'de_mobile_no',
       'de_product', 'de_type', 'de_telcoProvider', 'de_cdd_flag',
       'de_dateOfBirth', 'de_signUpAccNo', 'de_kycStatusfromCDD',
       'de_addressline1', 'de_city', 'de_province', 'de_barangay',
       'de_provinceCode', 'de_postalcode', 'de_mailingCity',
       'de_mailingPostalCode', 'de_mailingProvince', 'de_natureofwork',
       'de_employmentstatus', 'de_mobileOs', 'de_docType',
       'de_Minutestakentoonboard', 'de_companyName', 'de_industry',
       'de_device_dtl', 'de_Region_name', 'de_locationdfflag',
       'de_Onboarding_latitude', 'de_Onboarding_longitude',
       'de_loanjourney_latitude', 'de_loanjouney_longitude',
       'de_ApigeeLogs_latitude', 'de_ApigeeLogs_longitude',
       'de_loandfFlag', 'de_maritalStatus', 'de_dependentsCount',
       'de_monthlyIncome', 'de_startApplyDateTime', 'de_network_group',
       'de_Source', 'de_media_source', 'de_partner', 'de_campaign',
       'de_rnk', 'de_Age', 'de_SourceGroups', 'de_earliest_date',
       'de_months_difference', 'tx_cnt_total_transactions_5m',
       'tx_cnt_total_transactions_4m',
       'tx_cnt_total_transactions_bw_3m_and_4m',
       'tx_cnt_total_transactions_bw_4m_and_5m',
       'tx_cnt_cash_in_total_4m', 'tx_cnt_cash_in_ob2t_4m',
       'tx_cnt_cash_in_ot2t_4m', 'tx_amt_cash_in_total_4m',
       'tx_amt_cash_in_ob2t_4m', 'tx_amt_cash_in_ot2t_4m',
       'tx_cnt_cash_out_total_4m', 'tx_cnt_cash_out_billpay_4m',
       'tx_cnt_cash_out_cards_4m', 'tx_cnt_cash_out_t2ot_4m',
       'tx_cnt_cash_out_t2ob_4m', 'tx_amt_cash_out_total_4m',
       'tx_amt_cash_out_billpay_4m', 'tx_amt_cash_out_cards_4m',
       'tx_amt_cash_out_t2ot_4m', 'tx_amt_cash_out_t2ob_4m',
       'tx_avg_days_bt_trans_4m', 'tx_avg_days_bt_cash_in_trans_4m',
       'tx_avg_days_bt_cash_out_trans_4m', 'tx_med_days_bt_trans_4m',
       'tx_med_days_bt_cash_in_trans_4m',
       'tx_med_days_bt_cash_out_trans_4m', 'tx_deposit_accnt_cnt_4m',
       'tx_stash_accnt_opened_cnt_4m', 'tx_stash_accnt_closed_cnt_4m',
       'tx_stash_balance_4m', 'tx_td_accnt_opened_cnt_4m',
       'tx_td_accnt_completed_cnt_4m', 'tx_td_accnt_broken_cnt_4m',
       'tx_td_balance_4m', 'tx_tsa_balance_4m', 'tx_tsa_balance_5m',
       'tx_stash_balance_5m', 'tx_td_balance_5m',
       'tx_med_days_bw_new_dep_acct_open_4m',
       'tx_med_days_bw_td_acct_open_4m', 'tx_cnt_active_loans_end_4m',
       'tx_cnt_active_loans_end_5m', 'tx_cnt_rejected_loan_apps_4m',
       'tx_cnt_closed_loan_apps_4m', 'tx_cnt_active_loan_apps_4m',
       'tx_cnt_applied_loan_apps_4m', 'tx_cnt_approved_loan_apps_4m',
       'tx_cnt_disbursed_loan_apps_4m', 'tx_cnt_other_loan_apps_4m',
       'tx_avg_loan_request_tenure_4m', 'tx_avg_loan_approved_tenure_4m',
       'tx_avg_loan_request_amt_4m', 'tx_avg_loan_approved_amt_4m',
       'tx_avg_loan_disbursed_amt_4m', 'tx_cnt_fpd_10_4m',
       'tx_cnt_fpd_30_4m', 'tx_cnt_installments_paid_4m',
       'tx_amt_installment_paid_4m', 'appsflyer_id',
       'appsflyer_install_to_registration_minutes',
       'meng_app_launch_count_4m', 'meng_ql_sales_tile_count_4m',
       'meng_sil_sales_tile_count_4m', 'meng_ql_calculator_count_4m',
       'onboarding_to_first_cash_in_minutes', 'apg_latitude_4m',
       'apg_longitude_4m', 'apg_country_4m', 'apg_state_4m',
       'apg_county_4m', 'apg_city_4m', 'apg_municipality_4m',
       'apg_postcode_4m', 'apg_road_4m', 'apg_state_code_4m',
       'apg_result_type_4m', 'apg_formatted_4m', 'apg_address_line1_4m',
       'apg_address_line2_4m', 'meng_no_of_campaign_groups_overall',
       'meng_no_of_campaigns_push_overall',
       'meng_no_of_campaigns_email_overall',
       'meng_no_of_campaigns_sms_overall',
       'meng_no_of_campaigns_inapp_overall',
       'meng_no_times_communicated_overall',
       'meng_no_times_communicated_push_overall',
       'meng_no_times_communicated_email_overall',
       'meng_no_times_communicated_sms_overall',
       'meng_no_times_communicated_inapp_overall',
       'meng_total_campaign_group_conversion',
       'meng_no_of_impressions_freeluv', 'meng_push_impressions_freeluv',
       'meng_email_impressions_freeluv', 'meng_sms_impressions_freeluv',
       'meng_inapp_impressions_freeluv', 'meng_push_clicked_freeluv',
       'meng_email_clicked_freeluv', 'meng_sms_clicked_freeluv',
       'meng_inapp_clicked_freeluv',
       'meng_no_of_impressions_creditbuilder',
       'meng_push_impressions_creditbuilder',
       'meng_email_impressions_creditbuilder',
       'meng_sms_impressions_creditbuilder',
       'meng_inapp_impressions_creditbuilder',
       'meng_push_clicked_creditbuilder',
       'meng_email_clicked_creditbuilder',
       'meng_sms_clicked_creditbuilder',
       'meng_inapp_clicked_creditbuilder',
       'meng_no_of_impressions_pushqlfl',
       'meng_push_impressions_pushqlfl',
       'meng_email_impressions_pushqlfl', 'meng_sms_impressions_pushqlfl',
       'meng_inapp_impressions_pushqlfl', 'meng_push_clicked_pushqlfl',
       'meng_email_clicked_pushqlfl', 'meng_sms_clicked_pushqlfl',
       'meng_inapp_clicked_pushqlfl', 'meng_no_of_impressions_luvstash',
       'meng_push_impressions_luvstash',
       'meng_email_impressions_luvstash', 'meng_sms_impressions_luvstash',
       'meng_inapp_impressions_luvstash', 'meng_push_clicked_luvstash',
       'meng_email_clicked_luvstash', 'meng_sms_clicked_luvstash',
       'meng_inapp_clicked_luvstash', 'meng_no_of_impressions_rlflup',
       'meng_push_impressions_rlflup', 'meng_email_impressions_rlflup',
       'meng_sms_impressions_rlflup', 'meng_inapp_impressions_rlflup',
       'meng_push_clicked_rlflup', 'meng_email_clicked_rlflup',
       'meng_sms_clicked_rlflup', 'meng_inapp_clicked_rlflup',
       'meng_no_of_impressions_others', 'meng_push_impressions_others',
       'meng_email_impressions_others', 'meng_sms_impressions_others',
       'meng_inapp_impressions_others', 'meng_push_clicked_others',
       'meng_email_clicked_others', 'meng_sms_clicked_others',
       'meng_inapp_clicked_others', 'cl_day_120', 'cl_day_150',
       'cl_kycStatus', 'cl_creditScoreUpdated', 'cl_fraudScore',
       'cl_fraudScoreUpdated', 'cl_run_date', 'cl_brand',
       'cl_device__brand', 'cl_device__manufacturer', 'cl_device__model',
       'cl_telephony_info__network_operator_name',
       'cl_telephony_info__network_operator', 'cl_sim_operator_name',
       'cl_first_install_time', 'cl_pkg_Others', 'cl_pkg_action',
       'cl_pkg_adventure', 'cl_pkg_arcade', 'cl_pkg_art _ design',
       'cl_pkg_auto _ vehicles', 'cl_pkg_beauty', 'cl_pkg_board',
       'cl_pkg_books _ reference', 'cl_pkg_business', 'cl_pkg_card',
       'cl_pkg_casino', 'cl_pkg_casual', 'cl_pkg_comics',
       'cl_pkg_communication', 'cl_pkg_dating', 'cl_pkg_education',
       'cl_pkg_educational', 'cl_pkg_entertainment', 'cl_pkg_finance',
       'cl_pkg_food _ drink', 'cl_pkg_health _ fitness',
       'cl_pkg_lifestyle', 'cl_pkg_maps _ navigation', 'cl_pkg_medical',
       'cl_pkg_music', 'cl_pkg_music _ audio', 'cl_pkg_news _ magazines',
       'cl_pkg_personalization', 'cl_pkg_photography',
       'cl_pkg_productivity', 'cl_pkg_puzzle', 'cl_pkg_racing',
       'cl_pkg_roleplaying', 'cl_pkg_shopping', 'cl_pkg_simulation',
       'cl_pkg_social', 'cl_pkg_sports', 'cl_pkg_strategy',
       'cl_pkg_tools', 'cl_pkg_travel _ local', 'cl_pkg_trivia',
       'cl_pkg_videoplayers _ editors', 'cl_pkg_weather', 'cl_pkg_word',
       'cl_pkg_rated_for_12_plus', 'cl_pkg_rated_for_16_plus',
       'cl_pkg_rated_for_18_plus', 'cl_pkg_free', 'cl_pkg_pay',
       'cl_pkg_undefined']].copy()

In [18]:
df_filtered.columns.values

array(['customer_id', 'registration_date', 'tx_first_product',
       'tx_mau4m_active', 'tx_mau5m_active', 'tx_loanApplicationflag_4m',
       'submittedappliction4m', 'tx_loanApplicationflag_5m',
       'submittedappliction5m', 'tx_loanApprovedflag_5m'], dtype=object)

In [19]:
df1 = df1.merge(df_filtered, on = 'customer_id', how = 'inner')

In [20]:
df1.columns.values

array(['customer_id', 'onboarding_date', 'tx_first_product_x',
       'tx_first_product_user_segment', 'de_kyc_status',
       'de_educationdataflag', 'de_education_id', 'de_Education_Type',
       'de_clientdemographicflag', 'de_user_id', 'de_device_id',
       'de_created_dt', 'de_gender', 'de_birthplace', 'de_mobile_no',
       'de_product', 'de_type', 'de_telcoProvider', 'de_cdd_flag',
       'de_dateOfBirth', 'de_signUpAccNo', 'de_kycStatusfromCDD',
       'de_addressline1', 'de_city', 'de_province', 'de_barangay',
       'de_provinceCode', 'de_postalcode', 'de_mailingCity',
       'de_mailingPostalCode', 'de_mailingProvince', 'de_natureofwork',
       'de_employmentstatus', 'de_mobileOs', 'de_docType',
       'de_Minutestakentoonboard', 'de_companyName', 'de_industry',
       'de_device_dtl', 'de_Region_name', 'de_locationdfflag',
       'de_Onboarding_latitude', 'de_Onboarding_longitude',
       'de_loanjourney_latitude', 'de_loanjouney_longitude',
       'de_ApigeeLogs_latitude

In [21]:
print("The row count of both the dataframe should be same")
print(f"The shape of df filtered is: \t{df_filtered.shape}")
print(f"The shape of df1 is:\t{df1.shape}")

The row count of both the dataframe should be same
The shape of df filtered is: 	(76930, 10)
The shape of df1 is:	(76930, 281)


In [22]:
df1.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Customer_360_Data_Prep\Data\FinalData\20240925_LoanApplication_flag.csv", index = False)